## 2 Layer MLP
- With hidden layer of size 64 and dropout with 0.5 probability
- Softmax at last classification layer 
- Learning rate of 0.001
- Over 50-100 epochs

In [25]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model




encoder = load_model("../Models/AE/encoder_h5_wmse_200.keras")
X = np.load("../Data/LPS/classifier_lps_data.npy")
Y = np.load("../Data/LPS/classifier_lps_labels.npy")
print(X.shape)


(3315, 67499)


In [26]:
X = encoder(X)
print(X.shape)
del encoder

(3315, 200)


In [27]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=500,
    decay_rate=0.96,
    staircase=True
)

In [28]:
class MLPClassifier(tf.keras.Model):
    def __init__(self, hidden_size=64, dropout_rate=0.5, num_classes=2):
        super().__init__()
        self.classifier = tf.keras.Sequential([
           layers.BatchNormalization(),
            layers.Dense(hidden_size, activation='relu', kernel_initializer='he_normal'),
            layers.BatchNormalization(),  # BN after Dense but before activation is better
            layers.Dropout(dropout_rate),
            # Remove softmax - use from_logits=True in loss instead
            layers.Dense(num_classes)
        ])

    def call(self, inputs, training=True):
        return self.classifier(inputs, training=training)
     

In [29]:
# Split into train (60%), temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X.numpy(), Y, test_size=0.3, random_state=42, stratify=Y)
# Split temp into val (50% of temp = 20% total) and test (50% of temp = 20% total)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42, stratify=y_temp)
del X, X_temp, y_temp

In [30]:
print(f'Training data shape: {X_train.shape}')
print(f'Val data shape: {X_val.shape}')
print(f'Test data shape: {X_test.shape}')


Training data shape: (2320, 200)
Val data shape: (328, 200)
Test data shape: (667, 200)


In [31]:
# for layer in encoder.layers:
#     layer.trainable = True
model = MLPClassifier(hidden_size=64, dropout_rate=0.5)

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=4,
    restore_best_weights=True
)



model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # Important!
              metrics=['accuracy'],
            )

print("Training started....")
history = model.fit(
    X_train, y_train,
    epochs=50, batch_size=32,    
    validation_data=(X_val, y_val), 
    verbose=1,
    callbacks=[early_stopping,
            #    WandbMetricsLogger(),
            #    WandbModelCheckpoint("models.keras",save_best_only=True,monitor='accuracy')
               ]
)
# wandb.finish()
print("Training completed!")


Training started....
Epoch 1/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5000 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5000 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5000 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5000 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5000 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Training completed!


In [32]:
model.classifier.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization_2           │ (None, 200)            │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,050 (54.88 KB)

 Trainable params: 13,522 (52.82 KB)

 Non-trainable params: 528 (2.06 KB)

In [33]:
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay,precision_score,recall_score,f1_score
import matplotlib.pyplot as plt
loss, accuracy = model.evaluate(X_test, y_test)

y_pred_probs = model.predict(X_test)

y_pred = np.argmax(y_pred_probs, axis=1)

print(X_test)
print(y_pred_probs)
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
print(f'TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}')

if len(np.unique(y_test)) == 2:
    auc = roc_auc_score(y_test, y_pred_probs[:, 1])
    print(f'AUC: {auc}')
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs[:, 1])

    plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

# Confusion matrix plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
print(f'Test loss: {loss}')
print(f'Test Accuracy {accuracy}')
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = tn / (tn + fp)
false_positive_rate = fp / (fp + tn)
false_negative_rate = fn / (fn + tp)
true_positive_rate = tp / (tp + fn)  # Same as recall/sensitivity
negative_predictive_value = tn / (tn + fn)

print(f'Precision: {precision:.4f}')
print(f'Recall (Sensitivity/TPR): {recall:.4f}')
print(f'Specificity (TNR): {specificity:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'False Positive Rate (FPR): {false_positive_rate:.4f}')
print(f'False Negative Rate (FNR): {false_negative_rate:.4f}')
print(f'Negative Predictive Value (NPV): {negative_predictive_value:.4f}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4993 - loss: nan 
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[[ 0.1694518   0.07705009  0.03179234 ... -0.0232875  -0.05253173
   0.1329056 ]
 [ 0.16660307  0.08627603  0.02884777 ... -0.02539972 -0.03298884
   0.14725938]
 [ 0.2373464   0.11383488 -0.01293214 ...  0.09931312 -0.1578203
   0.2069871 ]
 ...
 [ 0.20179756  0.16238722  0.05749956 ...  0.02478223 -0.08555245
   0.17497899]
 [ 0.1221401  -0.00719697  0.15940358 ...  0.06335545 -0.10495584
   0.20531534]
 [ 0.17294478  0.11726473  0.11756313 ...  0.10296682 -0.06436104
   0.25149673]]
[[nan nan]
 [nan nan]
 [nan nan]
 ...
 [nan nan]
 [nan nan]
 [nan nan]]
TN: 333, FP: 0, FN: 334, TP: 0


ValueError: Input contains NaN.

In [ ]:
# model.encoder.save("../Models/Decoder/encoder_classifier.keras")
# model.classifier.save("../Models/Decoder/classifier.keras")
